**# Ejemplo de Árbol de decisión**

In [1]:
######################
# adquirir datos
######################

import pandas as pd
datos = pd.read_csv("./data/diabetes.csv") # dataframe

# Voy a categorizar en funcion de los casos sin diabetes (casos negativos)
casos_negativos = datos.sort_values(by='Outcome')
cantidad_casos_negativos = casos_negativos['Outcome'].value_counts(sort=False,ascending=True)[0]
casos_negativos = casos_negativos[:cantidad_casos_negativos]

In [ ]:
######################
# pre procesando datos metodo 1
######################
def mapearEdades(edad):
    if (edad >= 0 and edad <= 11): return 'niño'
    elif (edad >= 12 and edad <= 18): return 'adolescente'
    elif (edad >= 19 and edad <= 25): return 'joven'
    elif (edad >= 26 and edad <= 59): return 'adulto'
    elif (edad >= 60 and edad <= 70): return 'mayor'
    
datos['Age'] = datos['Age'].apply(mapearEdades)

# categorizar datos
# datos['Edades Categ'] = edades_categorizadas


In [ ]:
# Mapeo de glucosa en sangre
descriptor = casos_negativos['Glucose'].describe()
# Nota: Falta eliminar casos de glucosa=0

intervalos_glucosa = [
    0, 
    descriptor['mean'] - descriptor['std'], 
    descriptor['mean'] + descriptor['std'], 
    descriptor['max']
    ]
etiquetas_glucosa = ['baja', 'normal', 'alta']

datos['Glucose'] = pd.cut(datos['Glucose'], bins=intervalos_glucosa, labels=etiquetas_glucosa)
# print(datos)

In [10]:
# Analizo los datos en busca de ceros y faltantes
caracteristicas = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI','DiabetesPedigreeFunction']
cuenta = 0
for caract in caracteristicas:
    descriptor = casos_negativos[caract].describe()
    # Nota: Falta considerar casos de caracteristica=0
    cuenta_de_valores = casos_negativos[caract].value_counts()
    cantidad_ceros = cuenta_de_valores[0] if 0 in cuenta_de_valores else 0
    cuenta += cantidad_ceros
    print(f'cantidad de ceros en {caract}: {cantidad_ceros}')
print(f'cuenta:{cuenta}')

datos_ordenados = datos.sort_values(by=['BloodPressure'], ascending=True)
print(datos_ordenados)


cantidad de ceros en Glucose: 3
cantidad de ceros en BloodPressure: 19
cantidad de ceros en SkinThickness: 139
cantidad de ceros en Insulin: 236
cantidad de ceros en BMI: 9
cantidad de ceros en DiabetesPedigreeFunction: 0
cuenta:406
     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
347            3      116              0              0        0  23.5   
494            3       80              0              0        0   0.0   
222            7      119              0              0        0  25.2   
81             2       74              0              0        0   0.0   
78             0      131              0              0        0  43.2   
..           ...      ...            ...            ...      ...   ...   
549            4      189            110             31        0  28.5   
43             9      171            110             24      240  45.4   
177            0      129            110             46      130  67.1   
691           13      158  

In [ ]:
# Categorizo el resto de las caracteristicas segun el bloque anterior
caracteristicas = ['BloodPressure', 'SkinThickness', 'BMI', 'DiabetesPedigreeFunction']
caracteristicas_a_revisar = ['Insulin']

for caract in caracteristicas:
    descriptor = casos_negativos[caract].describe()
    # Nota: Falta considerar casos de caracteristica=0
    print(caract)
    intervalos = [
        0, 
        descriptor['mean'] - descriptor['std'], 
        descriptor['mean'] + descriptor['std'], 
        descriptor['max']
        ]
    etiquetas = ['baja', 'normal', 'alta']

    datos[caract] = pd.cut(datos[caract], bins=intervalos, labels=etiquetas)


In [ ]:
######################
# split datos
######################

# identificar característias y objetivo
caracteristicas = ["pelo", "estatura", "peso", "protector"]
X = datos[caracteristicas] # sub selección de columnas del dataframe
y = datos['quemado']

# dividir datos en trainset y testset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
######################
# modelar árbol de decisión
######################
from sklearn import tree

# definir modelo
dtree = tree.DecisionTreeClassifier(criterion="gini")  # id3
# entrenar modelo
dtree.fit(X_train, y_train)
# usar modelo
y_test_class = dtree.predict(X_test[caracteristicas])

In [ ]:
######################
# analizar performance
######################
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_test_class)
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dtree.classes_)
disp.plot()

In [ ]:
from matplotlib import image as pltimg
from matplotlib import pyplot as plt
import pydotplus

data = tree.export_graphviz(dtree, feature_names=caracteristicas,out_file=None)
graph = pydotplus.graph_from_dot_data(data)
graph.write_png("mi_arbol.png")

img = pltimg.imread("mi_arbol.png")
imgplot = plt.imshow(img)
plt.show()